In [ ]:
###

### Muffin vs Chihuahua · Tome III · Evaluation & Inference

#### Chapter I · Model Loading

In [ ]:
import yaml, torch, matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from pathlib import Path

In [ ]:
with open('../config/config.yaml') as f:
    cfg = yaml.safe_load(f)

test_dir = Path(cfg['data']['path']) / 'test'

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

test_ds = datasets.ImageFolder(test_dir, transform=transform)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet18(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load('../assets/muffin_vs_chihuahua.pth', map_location=device))
model = model.to(device); model.eval()

#### Chapter II · Accuracy & Confusion Matrix

In [ ]:
import numpy as np
y_true, y_pred = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        out = model(x)
        preds = torch.argmax(out, dim=1).cpu()
        y_true.extend(y.numpy())
        y_pred.extend(preds.numpy())

acc = (np.array(y_true) == np.array(y_pred)).mean()
print(f"Test accuracy: {acc:.2%}")

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=test_ds.classes)
disp.plot(cmap='Blues')
plt.show()

#### Chapter III · Solo Image Inference

In [ ]:
from PIL import Image

def predict(path):
    img = Image.open(path).convert('RGB')
    img_t = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(img_t)
        pred = torch.argmax(out, dim=1).item()
    return test_ds.classes[pred]

print(predict('../assets/images/sample_muffin.jpg'))